### Ensemble model on Xception, InceptionV3, and VGG19

In [1]:
! pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.models import load_model
path_to_models = "/content/drive/My Drive/data/FMD/saved_models/"

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
# get test dataset
materials = ['fabric','foliage','glass','leather','metal','paper','plastic','stone','water','wood']
IMG_SIZE = (224, 224)
path_to_test = "/content/drive/My Drive/data/FMD/test"
test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=path_to_test,
    labels='inferred',
    label_mode='categorical',
    class_names=materials,
    batch_size=4,
    image_size=IMG_SIZE,
    shuffle=False,
    seed=42,
    validation_split=None
)
AUTOTUNE = tf.data.AUTOTUNE
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 30 files belonging to 10 classes.


In [22]:
# function to multiply each model output with their corresponding weight 
def mul_weight(inputs, combo):
  w1 = w2 = w3 = 0.0
  if combo == 'combo1':
    w1 = 0.8
    w2 = 0.1
    w3 = 0.1
  elif combo == 'combo2':
    w1 = 0.7
    w2 = 0.2
    w3 = 0.1
  elif combo == 'combo3':
    w1 = 0.6
    w2 = 0.2
    w3 = 0.2
  elif combo == 'combo4':
    w1 = 0.5
    w2 = 0.3
    w3 - 0.2
  else: # default non-weighted average
    w1 = 1/3
    w2 = 1/3
    w3 = 1/3
  return tf.keras.layers.Lambda(lambda x: x*w1)(inputs[0]), tf.keras.layers.Lambda(lambda x: x*w2)(inputs[1]), tf.keras.layers.Lambda(lambda x: x*w3)(inputs[2])
    

In [6]:
# load the top 3 saved best models
model1 = load_model(path_to_models+"xception_sgd/best_xception_sgd_hypertune_gan.h5")
model1._name = 'xception'
model2 = load_model(path_to_models+"inception_sgd/best_inception_sgd_hypertune.h5")
model2._name = 'inceptionv3'
model3 = load_model(path_to_models+"hyperparameter_adam.h5")
model3._name = 'vgg19'
model_input = tf.keras.Input(shape=(224, 224, 3))

In [26]:
# function to create an ensemble model per combination of weights (5 in total)
def create_ensemble(combo):
  output1 = model1(model_input)
  output2 = model2(model_input)
  output3 = model3(model_input)
  outputs = [output1, output2, output3]
  output1, output2, output3 = mul_weight(outputs, combo) # multiply each model output with corresponding weights
  ensemble_output = tf.keras.layers.Average()([output1, output2, output3]) # and get the weighted average 
  ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output) # created ensemble model
  ensemble_model._name = 'ensemble'
  ensemble_model.compile(
          optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
          loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
          metrics=['accuracy']
      )
  return ensemble_model

In [27]:
# combo1: Xception: 0.8, InceptionV3: 0.1, VGG19: 0.1
ensemble_combo1 = create_ensemble('combo1')
eval_result1 = ensemble_combo1.evaluate(test_dataset)
print("[test loss, test accuracy]:", eval_result1)

8/8 [==============================] - 42s 4s/step - loss: 0.9277 - accuracy: 0.9000
[test loss, test accuracy]: [0.9277334809303284, 0.8999999761581421]


In [28]:
# combo2: Xception: 0.7, InceptionV3: 0.2, VGG19: 0.1
ensemble_combo2 = create_ensemble('combo2')
eval_result2 = ensemble_combo2.evaluate(test_dataset)
print("[test loss, test accuracy]:", eval_result2)

8/8 [==============================] - 38s 4s/step - loss: 0.9434 - accuracy: 0.9000
[test loss, test accuracy]: [0.9433820247650146, 0.8999999761581421]


In [29]:
# combo3: Xception: 0.6, InceptionV3: 0.2, VGG19: 0.2
ensemble_combo3 = create_ensemble('combo3')
eval_result3 = ensemble_combo3.evaluate(test_dataset)
print("[test loss, test accuracy]:", eval_result3)

8/8 [==============================] - 39s 4s/step - loss: 1.0186 - accuracy: 0.9000
[test loss, test accuracy]: [1.018591284751892, 0.8999999761581421]


In [30]:
# combo4: Xception: 0.5, InceptionV3: 0.3, VGG19: 0.2
ensemble_combo4 = create_ensemble('combo4')
eval_result4 = ensemble_combo4.evaluate(test_dataset)
print("[test loss, test accuracy]:", eval_result4)

8/8 [==============================] - 38s 4s/step - loss: 1.0649 - accuracy: 0.9000
[test loss, test accuracy]: [1.0648503303527832, 0.8999999761581421]


In [31]:
# default: Xception: 1/3, InceptionV3: 1/3, VGG19: 1/3, unweighted average
ensemble_default = create_ensemble('default')
eval_result_default = ensemble_default.evaluate(test_dataset)
print("[test loss, test accuracy]:", eval_result_default)

8/8 [==============================] - 38s 4s/step - loss: 1.1832 - accuracy: 0.8667
[test loss, test accuracy]: [1.183154821395874, 0.8666666746139526]


In [32]:
ensemble_combo1.save(path_to_models+"best_ensemble.h5") # best ensemble model is ensemble_combo1 according to test loss and accuracy